# Importing Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sea
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

# Preparing Dataset

In [ ]:
path = 'dataset'

# creating lists of filenames and disease outcome of each image
filenames, outcome = [], []
for label in os.listdir(path):
    for img in os.listdir(os.path.join(path,label)):
        filenames.append(os.path.join(path,label,img));
        outcome.append(label)

for i in range(len(filenames)):
    print(f'{filenames[i]} : {outcome[i]}')

# Data Preprocessing

In [ ]:
# Initialize lists for images and labels
X = []
y = []

# Define a consistent size for all images (e.g., 224x224 pixels)
image_size = (256, 256)

# Mapping outcomes to numerical labels
outcome_to_label = {
    "cataract": 0,
    "diabetic_retinopathy": 1,
    "glaucoma": 2,
    "other": 3  
}

# Convert images to numpy arrays and create labels
for i in range(len(filenames)):
    # Open image
    image = Image.open(filenames[i])
    # Convert image to RGB if not already
    if image.mode != 'RGB':
        image = image.convert('RGB')
    # Resize image to the defined size
    image = image.resize(image_size)
    # Convert image to numpy array
    X.append(np.array(image))
    # Append corresponding label
    y.append(outcome_to_label.get(outcome[i], 3))

In [ ]:
# conveting X and y in numpy array
X = np.array(X)
y = np.array(y)

In [ ]:
plt.imshow(X[0], cmap='gray')
plt.show()

In [ ]:
# splitting train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
# feature scaling
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Building Model

In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras import Sequential, models, layers
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import optimizers

In [ ]:
convolutional_base = ResNet50(weights = 'imagenet', include_top = False, input_shape = (256,256,3))
convolutional_base.summary()

In [ ]:
num_classes = 4
model = models.Sequential()
model.add(convolutional_base)
model.add(layers.Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.15, epochs=30)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Accuarcy = ', accuracy)

In [ ]:
# plot the loss value
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

# plot the accuracy value
plt.plot(history.history['acc'], label='train accuracy')
plt.plot(history.history['val_acc'], label='validation accuracy')
plt.legend()
plt.show()